# TPMI : Implementing Spherical Gaussian Mixture Model 

In [ ]:
# Importing Libraries
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
from IPython.display import clear_output as clr
from scipy.stats import multivariate_normal
from sklearn.cluster import KMeans as KM
import matplotlib as mpl
mpl.rc('image', cmap='gray')

In [ ]:
### Importing data
mat = scipy.io.loadmat('mnist_small.mat')
X = mat["X"]
y = mat["Y"]
print("Input shape", X.shape, ", Label shape", y.shape)

## Defining the GMM Class

In [ ]:
## Defining the Class of GMM Model
class GMM_spherical:
    '''
    ||========================================================= Model GMM ==========================================================||
    ||___________________________________________________Developed by Abhishek Kumar________________________________________________||
    ||______________________________________________________________________________________________________________________________||
    
    This is Spherical GMM Model with both batch and stepwise EM algorithm implemented as fit and partial_fit for function respectively.
    '''
    def __init__(self, n_components = 10, max_iter = 200):
        
        self.n_components = n_components
        self.means = None
        self.vars = None
        self.pis = None
        self.max_iter = max_iter
        self.idls = [] # Incomplete data log likelihoods
        self.nums = []
    
    def InitVars(self, D):
        self.pis = np.ones((self.n_components,1))/self.n_components
        self.means = np.random.rand(self.n_components, D)
        self.vars = np.array([1])
        
    def E_step(self, X):
        '''
        X : N x D
        '''
        N, D = X.shape
        gamma = np.zeros((X.shape[0], self.n_components))
        for j in range(self.n_components):
            
            delta = X-self.means[j,:] # NxD 
            expo = -0.5*(np.sum(delta**2, axis = 1))/(self.vars)
            gamma[:,j] = expo+np.log(self.pis[j])-(D/2)*np.log(2*np.pi*self.vars)
        
        gamma = np.exp((gamma-gamma.max(axis = 1).reshape(-1,1)))
        gamma = gamma/np.sum(gamma, axis = 1).reshape(-1,1)
        
        return gamma
    
    def M_step(self, X, gammas):
        '''
        X : NxD
        gammas : NxK
        '''
        N, D = X.shape
        K = gammas.shape[1]
        Nms = np.sum(gammas, axis = 0).reshape(K,1) # Kx1
        
        #pi
        self.pis = Nms/N
        # mu
        numerator = np.dot(gammas.T, X) # KxD
        self.means = numerator/Nms
        # var
        prod = np.zeros((N,K))
        for j in range(K):
            delta = X - self.means[j,:] # NxD
            vec = np.sum(delta**2, axis = 1) # Nx1
            prod[:,j] = vec
        
        prod = prod*gammas
        self.vars = np.sum(prod)/(N*D)
        
    def get_idl(self, X):
        N, D = X.shape
        X_max = X.max()
        likhd = 0
        gamma = np.zeros((N, self.n_components))
        for j in range(self.n_components):    
            delta = X-self.means[j,:] # NxD 
            expo = -0.5*(np.sum(delta**2, axis = 1))/(self.vars)
            gamma[:,j] = expo+np.log(self.pis[j])-(D/2)*np.log(2*np.pi*self.vars)
            
        gammas = self.E_step(X)
        mul = np.sum(gammas*gamma)
        
        return mul
    
    def fit(self, X):
        '''
        This is Batch EM version
        '''
        N, D = X.shape
        self.InitVars(D)
        old_idl = self.get_idl(X)
        new_idl = old_idl + 100
        for i in range(self.max_iter):
            gammas = self.E_step(X)
            self.M_step(X, gammas)
            new_idl = self.get_idl(X)
            print(new_idl)
            clr(wait = True)
            self.idls.append(new_idl)
            self.nums.append(i*N)
            diff = abs(new_idl - old_idl)
            if(diff < 1):
                break
            old_idl = new_idl+0.000001
            
        return self.nums, self.idls 
    
    def partial_fit(self, X, batch_size = 100, kr = 0.55, init_me = True, km_init = False):
        '''
        This is the stepwise EM version
        '''
        N, D = X.shape
        if(init_me):
            self.InitVars(D)
            if(km_init):
                kmmodel = KM(n_clusters = self.n_components)
                kmmodel.fit(X)
                self.means = kmmodel.cluster_centers_
        else:
            assert(self.means.shape[1] == D)

        old_idl = -999
        new_idl = old_idl + 100
        old_idl_mini = 0
        new_idl_mini = old_idl_mini + 100
        diff = 1000    
        np.random.shuffle(X)
        old_lrate = 0
        n_batches = (N//batch_size)
        gammas = self.E_step(X)
        self.M_step(X, gammas)
        
        for i in range(self.max_iter):
            l_rate = ((i//n_batches)+1)**(-kr)
            ul = i%batch_size
            ll = min(ul+batch_size, X.shape[0])
            mini_X = X[ul:ll,:]
            if(mini_X.shape[0] == 0):
                continue 

            pis = self.pis.copy()
            mus = self.means.copy()
            var = self.vars.copy()

            gammas = self.E_step(mini_X)
            self.M_step(mini_X, gammas)

            self.pis = pis*(1-l_rate) + l_rate*(self.pis)
            self.means = mus*(1-l_rate) + l_rate*(self.means)
            self.vars = var*(1-l_rate) + l_rate*(self.vars)
                
            if(i%n_batches==0):
                new_idl = self.get_idl(X)
                print(new_idl, diff)
                clr(wait = True)
                self.idls.append(new_idl)
                self.nums.append(i*batch_size)
                diff = abs(new_idl - old_idl)
                if(diff < 1):
                    break
                old_idl = new_idl+0.000001
            
            
        return self.nums, self.idls 

## Function for running the model as EM, sEM, sEM with kmeans init.

In [ ]:
def calc_all(K, F, bsize  =2000):
    
    ##  Batch EM 
    model = GMM_spherical(n_components=K)
    his_batch = model.fit(Xd)
    fig, ax = plt.subplots(F,K//F, figsize = [K//F,F+1])
    for i in range(K):
        if(F==1):
            ax[i//F].imshow((model.means[i]*stdX + meanX).reshape(28,28))
        else: 
            ax[i%F][i//F].imshow((model.means[i]*stdX + meanX).reshape(28,28))
    fig.suptitle("EM at K = " + str(K))
    plt.savefig("GMMbEM_"+str(K)+".png")
    plt.show()
    
    ## Stepwise EM (sEM)
    model = GMM_spherical(n_components=K)
    his_stepwise = model.partial_fit(Xd, batch_size = bsize, kr = 0.55, init_me = True, km_init = False)
    fig, ax = plt.subplots(F,K//F, figsize = [K//F,F+1])
    for i in range(K):
        if(F == 1):
            ax[i//F].imshow((model.means[i]*stdX + meanX).reshape(28,28))
        else:    
            ax[i%F][i//F].imshow((model.means[i]*stdX + meanX).reshape(28,28))
    fig.suptitle("sEM at K = " + str(K))
    plt.savefig("GMMsEM_"+str(K)+".png")
    plt.show()
    
    ## Stepwise EM with Kmeans init (skEM)
    model = GMM_spherical(n_components=K)
    his_stepwise_km = model.partial_fit(Xd, batch_size = bsize, kr = 0.55, init_me = True, km_init = True)
    fig, ax = plt.subplots(F,K//F, figsize = [K//F,F+1])
    for i in range(K):
        if(F==1):
            ax[i//F].imshow((model.means[i]*stdX + meanX).reshape(28,28))
        else:
            ax[i%F][i//F].imshow((model.means[i]*stdX + meanX).reshape(28,28))
    fig.suptitle("sEM at K = " + str(K) + "with Kmeans init")
    plt.savefig("GMMskEM_"+str(K)+".png")
    plt.show()
    
    '''
    Comparing the log likelihoods ...............
    '''
    plt.plot(his_batch[0],his_batch[1], label  = "EM")
    plt.plot(his_stepwise[0],his_stepwise[1], label  = "sEM")
    plt.plot(his_stepwise_km[0],his_stepwise_km[1], label = "sEM Kmean Init")
    plt.title("K = "+str(K)+" Likelihoods")
    plt.legend()
    plt.savefig("Compare" + str(K)+".png")
    plt.show()

## Normalizing the data for numerical stabilitiy

In [ ]:
meanX = X.mean()
stdX = X.std()
min_X = X.min()
max_X = X.max()

In [ ]:
# Xd = (X - meanX)/stdX
Xd = (X - min_X)/(max_X - min_X)

## Running the GMM at different K values for Mnist data.

Run all four cells below, This might take few minutes to run. I suggest you go drink a tea relax, then come back and look at the plots :)

In [ ]:
K = 5
F = K//5
calc_all(K,F)

In [ ]:
K = 10
F = K//5
calc_all(K,F)

In [ ]:
K = 15
F = K//5
calc_all(K,F)

In [ ]:
K = 20
F = K//5
calc_all(K,F)

### Thankyou

Abhishek Kumar

18111002